# Overfitting and regularization (with ``gluon``)

Now that we've built a [logistic regression model from scratch](http://5-softmax-reression-scratch.ipynb), let's make this more efficient with ``gluon``.

In [1]:
from __future__ import print_function
import mxnet as mx
from mxnet import nd, autograd
from mxnet import gluon
import numpy as np
ctx = mx.cpu()

## The MNIST Dataset

In [2]:
mnist = mx.test_utils.get_mnist()
num_examples = 1000
batch_size = 64
train_data = mx.io.NDArrayIter(
    mnist["train_data"][:num_examples], 
    mnist["train_label"][:num_examples], 
    batch_size, shuffle=True)
test_data = mx.io.NDArrayIter(
    mnist["test_data"][:num_examples], 
    mnist["test_label"][:num_examples], 
    batch_size, shuffle=True)

## Multiclass Logistic Regression

In [3]:
net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Dense(10))

## Parameter initialization


In [4]:
net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

## Softmax Cross Entropy Loss

In [5]:
loss = gluon.loss.SoftmaxCrossEntropyLoss()

## Optimizer

In [6]:
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.01, 'wd': 0.00})

## Evaluation Metric

In [7]:
def evaluate_accuracy(data_iterator, net):
    numerator = 0.
    denominator = 0.
    data_iterator.reset()
    for i, batch in enumerate(data_iterator):
        data = batch.data[0].as_in_context(ctx).reshape((-1,784))
        label = batch.label[0].as_in_context(ctx)
        label_one_hot = nd.one_hot(label, 10)
        output = net(data)        
        predictions = nd.argmax(output, axis=1)
        numerator += nd.sum(predictions == label)
        denominator += data.shape[0]
    return (numerator / denominator).asscalar()

## Execute training loop

In [8]:
epochs = 1000
moving_loss = 0.

for e in range(epochs):
    train_data.reset()
    for i, batch in enumerate(train_data):
        data = batch.data[0].as_in_context(ctx).reshape((-1,784))
        label = batch.label[0].as_in_context(ctx)
        with autograd.record():
            output = net(data)
            cross_entropy = loss(output, label)
        cross_entropy.backward()
        trainer.step(data.shape[0])
        
        ##########################
        #  Keep a moving average of the losses
        ##########################
        if i == 0:
            moving_loss = nd.mean(cross_entropy).asscalar()
        else:
            moving_loss = .99 * moving_loss + .01 * nd.mean(cross_entropy).asscalar()
            
    test_accuracy = evaluate_accuracy(test_data, net)
    train_accuracy = evaluate_accuracy(train_data, net)
    if e % 100 == 99:
        print("Completed epoch %s. Loss: %s, Train_acc %s, Test_acc %s" % 
              (e+1, moving_loss, train_accuracy, test_accuracy))           

Completed epoch 100. Loss: 0.442401012278, Train_acc 0.902344, Test_acc 0.811523
Completed epoch 200. Loss: 0.3228339456, Train_acc 0.930664, Test_acc 0.827148
Completed epoch 300. Loss: 0.265595163307, Train_acc 0.941406, Test_acc 0.835938
Completed epoch 400. Loss: 0.228991834718, Train_acc 0.952148, Test_acc 0.837891
Completed epoch 500. Loss: 0.202418237662, Train_acc 0.959961, Test_acc 0.847656
Completed epoch 600. Loss: 0.181711741699, Train_acc 0.96582, Test_acc 0.845703
Completed epoch 700. Loss: 0.164875624125, Train_acc 0.969727, Test_acc 0.841797
Completed epoch 800. Loss: 0.15082833116, Train_acc 0.974609, Test_acc 0.838867
Completed epoch 900. Loss: 0.138909899877, Train_acc 0.975586, Test_acc 0.837891
Completed epoch 1000. Loss: 0.128667225458, Train_acc 0.979492, Test_acc 0.838867


## Conclusion

Now let's take a look at how to implement modern neural networks. 

For whinges or inquiries, [open an issue on  GitHub.](https://github.com/zackchase/mxnet-the-straight-dope)

In [9]:
params = net.collect_params()

In [10]:
params['sequential0_dense0_weight'].wd_mult

1.0

In [11]:
p.wd_mult

NameError: name 'p' is not defined